In [ ]:
# Extract videos
# !unrar e "/content/drive/MyDrive/ML_Datasets/UCF101.rar" "/content/drive/MyDrive/ML_Datasets/UCF101/"

In [1]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from tensorflow.keras.utils import to_categorical
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob   # reading files
from tqdm import tqdm
import os

Store the name of videos in a dataframe

In [2]:
train_test_list_dir = "/Users/daviddela/Intro to AI/AI Final Project/archive"

In [3]:
# open the .txt file which have names of training videos
# train_list = os.path.join(train_test_list_dir, "trainlist01.txt")

# f = open(train_list, "r")
# temp = f.read()
# videos = temp.split('\n')

# # creating a dataframe having video names
# train = pd.DataFrame()
# train['video_name'] = videos
# train = train[:-1]
# train.head()

In [4]:
# Loop through the entire set of videos and generate train and test lists
data_dir = "/Users/daviddela/Intro to AI/AI Final Project/archive"

actions_list = [
    'barbell_biceps_curl', 'bench_press', 'chest_fly_machine', 'deadlift', 'decline_bench_press', 'hammer_curl', 
    'hip_thrust', 'incline_bench_press', 'lat_pulldown', 'lateral_raise', 'leg_extension', 'leg_raises', 'plank', 
    'pull_up', 'push_up', 'romanian_deadlift', 'russian_twist', 'shoulder_press', 'squat', 't_bar_row', 'tricep_dips', 
    'tricep_pushdown'
]



In [10]:
# actions_list = actions_list[:-2]
print(actions_list)

['barbell_biceps_curl', 'bench_press', 'chest_fly_machine', 'deadlift', 'decline_bench_press', 'hammer_curl', 'hip_thrust', 'incline_bench_press', 'lat_pulldown', 'lateral_raise', 'leg_extension', 'leg_raises', 'plank', 'pull_up', 'push_up', 'romanian_deadlift', 'russian_twist', 'shoulder_press', 'squat', 't_bar_row', 'tricep_dips', 'tricep_pushdown']


In [11]:
train_list = []
test_list = []
split_ratio = 0.8

for action in actions_list:
    action_videos = os.listdir(os.path.join(data_dir, action))
    split_idx = int(split_ratio * len(action_videos))
    
    train_vids = action_videos[:split_idx]
    test_vids = action_videos[split_idx:]

    train_list.extend(train_vids)
    test_list.extend(test_vids)

In [12]:
train = pd.DataFrame()
train['video_name'] = train_list
train.head()

,video_name
0,barbell biceps curl_46.mp4
1,barbell biceps curl_52.mp4
2,barbell biceps curl_53.mp4
3,barbell biceps curl_47.mp4
4,barbell biceps curl_51.mp4


In [13]:
test = pd.DataFrame()
test['video_name'] = test_list
test.head()

,video_name
0,barbell biceps curl_17.mp4
1,barbell biceps curl_15.mp4
2,barbell biceps curl_1.mp4
3,barbell biceps curl_29.mp4
4,barbell biceps curl_28.mp4


In [14]:
# Modifying the code to extract just the action name
train_video_tag = []
for i in range(train.shape[0]):
    # Splitting the filename on underscore and excluding the last part (number and file extension)
    action_name_parts = train['video_name'][i].split('_')[:-1]
    # Joining the parts back together to form the complete action name
    action_name = ' '.join(action_name_parts)
    train_video_tag.append(action_name)

train['tag'] = train_video_tag


In [15]:
train.head()

,video_name,tag
0,barbell biceps curl_46.mp4,barbell biceps curl
1,barbell biceps curl_52.mp4,barbell biceps curl
2,barbell biceps curl_53.mp4,barbell biceps curl
3,barbell biceps curl_47.mp4,barbell biceps curl
4,barbell biceps curl_51.mp4,barbell biceps curl


In [16]:
# Modifying the code to extract just the action name for test videos
test_video_tag = []
for i in range(test.shape[0]):
    # Splitting the filename on underscore and excluding the last part (number and file extension)
    action_name_parts = test['video_name'][i].split('_')[:-1]
    # Joining the parts back together to form the complete action name
    action_name = ' '.join(action_name_parts)
    test_video_tag.append(action_name)

test['tag'] = test_video_tag

In [17]:
test.head()

,video_name,tag
0,barbell biceps curl_17.mp4,barbell biceps curl
1,barbell biceps curl_15.mp4,barbell biceps curl
2,barbell biceps curl_1.mp4,barbell biceps curl
3,barbell biceps curl_29.mp4,barbell biceps curl
4,barbell biceps curl_28.mp4,barbell biceps curl


Extract frames from training videos to be used in training

In [18]:
# Define the directory where frames will be saved
train_frames_dir = os.path.join(data_dir, "train_frames")

# Ensure the directory exists (creates if it doesn't, does nothing if it does)
os.makedirs(train_frames_dir, exist_ok=True)

In [19]:
from tqdm import tqdm
import os
import cv2

for i in tqdm(range(train.shape[0])):
    videoFile = train['video_name'][i]
    action_tag = train['tag'][i]  # Assuming 'tag' column contains the action name

    # Skip if the action tag is not in the predefined list of actions
    if action_tag not in actions_list:
        continue

    vid_path = os.path.join(data_dir, action_tag, videoFile)

    # Check if the video file exists
    if not os.path.exists(vid_path):
        print(f"File not found: {vid_path}")
        continue

    cap = cv2.VideoCapture(vid_path)

    if not cap.isOpened():
        print(f"Could not open video: {vid_path}")
        continue

100%|████████████████████████████████████████| 511/511 [00:00<00:00, 713.67it/s]


In [22]:
# Frame extraction and saving process
frame_extraction_interval = 5
for i in tqdm(range(train.shape[0])):
    videoFile = train['video_name'][i]
    action_tag = train['tag'][i]  # Ensure this correctly represents the action/category

    if action_tag not in actions_list:
        continue

    vid_path = os.path.join(data_dir, action_tag, videoFile)
    if not os.path.exists(vid_path):
        print(f"File not found: {vid_path}")
        continue

    cap = cv2.VideoCapture(vid_path)
    if not cap.isOpened():
        print(f"Could not open video: {vid_path}")
        continue

    frameRate = cap.get(5)  # frame rate
    currFrame = 0

    while(cap.isOpened()):
        ret, frame = cap.read()
        if not ret:
            break

        # Save frames at a regular interval, e.g., one frame per second
        if currFrame % int(frameRate* frame_extraction_interval) == 0:
            img = f"{action_tag}_frame{currFrame}.jpg"
            filename = os.path.join(train_frames_dir, img)
            cv2.imwrite(filename, frame)

        currFrame += 1

    cap.release()
print("Done!")

100%|█████████████████████████████████████████| 511/511 [00:33<00:00, 15.12it/s]

Done!


Save the name of these frames with their corresponding tag in a .csv file

In [27]:
import re
images = os.listdir(train_frames_dir)
train_image = []
train_class = []

for i in tqdm(range(len(images))):
    image_name = images[i]

    # Check if filename starts with a number
    if re.match(r'^\d', image_name):
        continue  # Skip this file

    train_image.append(image_name)
    train_class.append(image_name.split('_')[0])

# Storing the images and their class in a dataframe
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

# Converting the dataframe into a CSV file
train_data.to_csv(os.path.join(data_dir, 'train_new.csv'), header=True, index=False)

100%|████████████████████████████████| 63411/63411 [00:00<00:00, 1981722.62it/s]


In [28]:
print(train_image)
print(train_class)

['plank_frame3114.jpg', 'squat_frame164.jpg', 'plank_frame5565.jpg', 'chest fly machine_frame86.jpg', 'romanian deadlift_frame2129.jpg', 'romanian deadlift_frame1420.jpg', 'plank_frame1703.jpg', 'pull up_frame479.jpg', 'plank_frame654.jpg', 'leg extension_frame468.jpg', 'shoulder press_frame554.jpg', 'russian twist_frame1290.jpg', 'shoulder press_frame232.jpg', 'chest fly machine_frame379.jpg', 'plank_frame132.jpg', 'romanian deadlift_frame1346.jpg', 'plank_frame1065.jpg', 'squat_frame11.jpg', 'russian twist_frame863.jpg', 'plank_frame5203.jpg', 'tricep dips_frame468.jpg', 'squat_frame602.jpg', 'plank_frame3672.jpg', 'plank_frame2578.jpg', 'russian twist_frame877.jpg', 't bar row_frame709.jpg', 'plank_frame5217.jpg', 'plank_frame4109.jpg', 'squat_frame616.jpg', 'plank_frame3666.jpg', 'russian twist_frame1284.jpg', 'shoulder press_frame226.jpg', 'plank_frame126.jpg', 'plank_frame1071.jpg', 'romanian deadlift_frame1352.jpg', 'plank_frame1717.jpg', 'romanian deadlift_frame1434.jpg', 'plan

Training the Video Classification Model

In [29]:
import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [31]:
# Read in the csv file containing training images and their classes
train = pd.read_csv(os.path.join(data_dir, 'train_new.csv'))
train.head()

,image,class
0,plank_frame3114.jpg,plank
1,squat_frame164.jpg,squat
2,plank_frame5565.jpg,plank
3,chest fly machine_frame86.jpg,chest fly machine
4,romanian deadlift_frame2129.jpg,romanian deadlift


In [32]:
X = []

for i in range(train.shape[0]):
  img = train['image'][i]
  img = image.load_img(os.path.join(train_frames_dir, img), target_size=(224,224,3))
  img_arr = image.img_to_array(img)
  img_arr /= 255.0
  X.append(img_arr)

X = np.array(X)
X.shape

(22612, 224, 224, 3)

In [33]:
# Create a validation set
y = train['class']

# Creating the train and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [34]:
X_train.shape, X_test.shape

((18089, 224, 224, 3), (4523, 224, 224, 3))

In [35]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

Define architecture of the video classification model

In [36]:
# # creating the base model of pre-trained VGG16 model
# base_model = VGG16(weights='imagenet', include_top=False)

base_model = VGG16(include_top=False, weights="imagenet", input_shape=(224,224,3))
base_model.summary()

2023-12-03 20:03:45.402191: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-12-03 20:03:45.403285: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-12-03 20:03:45.403301: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-12-03 20:03:45.404157: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-03 20:03:45.404701: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [37]:
outputs = base_model.get_layer("block5_pool").output
feature_extractor = Model(inputs=base_model.input, outputs=outputs)

In [42]:
with tf.device('/cpu:0'):
    X_train = feature_extractor.predict(X_train)

2023-12-03 20:10:48.371850: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


566/566 [==============================] - 2990s 5s/step


In [43]:
# extracting features for training frames
#X_train = feature_extractor.predict(X_train)
X_train.shape

(18089, 7, 7, 512)

In [44]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

142/142 [==============================] - 105s 734ms/step


(4523, 7, 7, 512)

In [45]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(X_train.shape[0], 7*7*512)
X_test = X_test.reshape(X_test.shape[0], 7*7*512)

In [46]:
# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

In [47]:
# shape of images
X_train.shape

(18089, 25088)

In [52]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(21, activation='softmax'))

In [53]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [54]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [55]:
# training the model
model.fit(X_train, y_train, epochs=200, validation_data=(
    , y_test), callbacks=[mcp_save], batch_size=128)

Epoch 1/200
142/142 [==============================] - ETA: 0s - loss: 32.1858 - accuracy: 0.3736

/Users/daviddela/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


142/142 [==============================] - 10s 53ms/step - loss: 32.1858 - accuracy: 0.3736 - val_loss: 18.2553 - val_accuracy: 0.5501
Epoch 2/200
142/142 [==============================] - 7s 47ms/step - loss: 25.5725 - accuracy: 0.6071 - val_loss: 3.9420 - val_accuracy: 0.8211
Epoch 3/200
142/142 [==============================] - 7s 47ms/step - loss: 11.1447 - accuracy: 0.8051 - val_loss: 0.8495 - val_accuracy: 0.9637
Epoch 4/200
142/142 [==============================] - 7s 47ms/step - loss: 4.4313 - accuracy: 0.9120 - val_loss: 0.0916 - val_accuracy: 0.9951
Epoch 5/200
142/142 [==============================] - 7s 46ms/step - loss: 2.2592 - accuracy: 0.9519 - val_loss: 0.0129 - val_accuracy: 0.9989
Epoch 6/200
142/142 [==============================] - 7s 46ms/step - loss: 1.1850 - accuracy: 0.9741 - val_loss: 1.2249e-04 - val_accuracy: 1.0000
Epoch 7/200
142/142 [==============================] - 7s 46ms/step - loss: 0.8204 - accuracy: 0.9828 - val_loss: 0.0000e+00 - val_accuracy

142/142 [==============================] - 10s 68ms/step - loss: 2.1743 - accuracy: 0.9987 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 57/200
142/142 [==============================] - 10s 67ms/step - loss: 3.0093 - accuracy: 0.9985 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 58/200
142/142 [==============================] - 10s 72ms/step - loss: 1.4155 - accuracy: 0.9992 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 59/200
142/142 [==============================] - 10s 73ms/step - loss: 2.5860 - accuracy: 0.9987 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 60/200
142/142 [==============================] - 9s 65ms/step - loss: 9.8924 - accuracy: 0.9965 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 61/200
142/142 [==============================] - 10s 69ms/step - loss: 5.0413 - accuracy: 0.9978 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 62/200
142/142 [==============================] - 9s 66ms/step - loss: 3.2239 - accuracy: 0.9988 - val_loss:

142/142 [==============================] - 12s 84ms/step - loss: 2.6246 - accuracy: 0.9993 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 111/200
142/142 [==============================] - 12s 82ms/step - loss: 3.1619 - accuracy: 0.9996 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 112/200
142/142 [==============================] - 8s 59ms/step - loss: 6.2436 - accuracy: 0.9991 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 113/200
142/142 [==============================] - 8s 55ms/step - loss: 9.2100 - accuracy: 0.9994 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 114/200
142/142 [==============================] - 11s 77ms/step - loss: 6.3984 - accuracy: 0.9989 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 115/200
142/142 [==============================] - 12s 83ms/step - loss: 4.1032 - accuracy: 0.9996 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 116/200
142/142 [==============================] - 12s 83ms/step - loss: 4.7572 - accuracy: 0.9993 - val

142/142 [==============================] - 15s 105ms/step - loss: 3.0254 - accuracy: 0.9998 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 165/200
142/142 [==============================] - 14s 96ms/step - loss: 2.4973 - accuracy: 0.9998 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 166/200
142/142 [==============================] - 13s 93ms/step - loss: 8.9568 - accuracy: 0.9994 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 167/200
142/142 [==============================] - 13s 89ms/step - loss: 8.8168 - accuracy: 0.9997 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 168/200
142/142 [==============================] - 13s 89ms/step - loss: 3.5785 - accuracy: 0.9998 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 169/200
142/142 [==============================] - 12s 88ms/step - loss: 8.1868 - accuracy: 0.9995 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 170/200
142/142 [==============================] - 12s 86ms/step - loss: 0.6337 - accuracy: 0.9999 - 

In [61]:
y_test.shape

(4523, 21)

In [82]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import KFold
import numpy as np

# Assuming inputs and targets are your data and labels
inputs = np.array([X_train])  
targets = np.array([y_train])  

In [83]:
num_samples = inputs.shape[0]  
num_folds = min(5, num_samples)


In [85]:
print("Shape of inputs:", inputs.shape)
print("Shape of targets:", targets.shape)

Shape of inputs: (1, 18089, 25088)
Shape of targets: (1, 18089, 21)


In [86]:
inputs = np.reshape(inputs, (18089, 25088))  # Reshape to 2D array
targets = np.reshape(targets, (18089, 21))    # Reshape to 2D array

print("Reshaped Inputs:", inputs.shape)
print("Reshaped Targets:", targets.shape)

Reshaped Inputs: (18089, 25088)
Reshaped Targets: (18089, 21)


### Model Evaluation

In [91]:
from keras.models import load_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred_classes = np.argmax(X_test, axis=1) # Convert probabilities to class labels
y_test_classes = np.argmax(y_test, axis=1) # Convert one-hot encoded test labels to class labels

accuracy = accuracy_score(y_test_classes, y_pred_classes)
precision = precision_score(y_test_classes, y_pred_classes, average='macro')
recall = recall_score(y_test_classes, y_pred_classes, average='macro')
f1 = f1_score(y_test_classes, y_pred_classes, average='macro')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

ValueError: Shape of passed values is (4523, 1), indices imply (4523, 21)

In [ ]:
# 0. Install and Import Dependencies

#!pip install mediapipe opencv-python



import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Camera could not be opened.")
else:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            print("Error: Frame not captured.")
            break

        cv2.imshow('Mediapipe Feed', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


# 1. Make Detections

cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=5, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,90,230), thickness=5, circle_radius=2)
                                 )
        print(results)
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

mp_drawing.DrawingSpec??

mp_pose.POSE_CONNECTIONS

# 2. Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass


        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

len(landmarks)

for lndmrk in mp_pose.PoseLandmark:
    print(lndmrk)

landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility

landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

# 3. Calculate Angles

def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle >180.0:
        angle = 360-angle

    return angle

shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

shoulder, elbow, wrist

calculate_angle(shoulder, elbow, wrist)

tuple(np.multiply(elbow, [640, 480]).astype(int))

cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)

            # Visualize angle
            cv2.putText(image, str(angle),
                           tuple(np.multiply(elbow, [640, 480]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )

        except:
            pass


        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# 4. Curl Counter

cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)

            # Visualize angle
            cv2.putText(image, str(angle),
                           tuple(np.multiply(elbow, [640, 480]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )

            # Curl counter logic
            if angle > 160:
                stage = "down"
            if angle < 30 and stage =='down':
                stage="up"
                counter +=1
                print(counter)

        except:
            pass

        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)


        # Box dimensions - adjust these to make the box bigger
        box_width = 300  # increased width
        box_height = 100  # increased height

        # Rectangle transparency factor (between 0 and 1)
        alpha = 0.9

        # Rectangle color
        rectangle_color = (245,117,16)

        # Coordinates of the rectangle's top left and bottom right corners
        rectangle_top_left = (0,0)
        rectangle_bottom_right = (box_width, box_height)

        # Create an overlay image with the same size as the frame
        overlay = image.copy()

        # Draw a solid rectangle on the overlay
        cv2.rectangle(overlay, rectangle_top_left, rectangle_bottom_right, rectangle_color, -1)

        # Blend the overlay with the original image
        cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0, image)


        # Box dimensions
       #box_width = 225
        #box_height = 73

        # Set the rectangle background to blue
        cv2.rectangle(image, (0,0), (box_width, box_height), (245,117,16), -1)

        # Define the starting position for 'REPS' label and value
        reps_label_x = 15
        reps_value_x = reps_label_x + 10
        # Define the starting position for 'STAGE' label and value
        stage_label_x = 125  # Adjusted for spacing
        stage_value_x = stage_label_x + 10

        # Define vertical positions for labels and values
        label_y = 12
        value_y = box_height - 10

        # Reps text
        cv2.putText(image, 'REPS', (reps_label_x, label_y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter),
                    (reps_value_x, value_y),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

        # Stage text
        cv2.putText(image, 'STAGE', (stage_label_x, label_y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage,
                    (stage_value_x, value_y),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# For push ups

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

# Enhanced pushup detection logic
counter = 0
stage = 'up'  # Start in the 'up' position

# Capture video from camera
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Camera could not be opened.")
else:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Error: Frame not captured.")
            break

        # Flip the frame horizontally for a laterally correct mirror view
        frame = cv2.flip(frame, 1)

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates for shoulder, elbow, and wrist
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            # Calculate arm angle
            arm_angle = calculate_angle(shoulder, elbow, wrist)

            # Pushup detection and counting logic
            if stage == 'up' and arm_angle < 90:  # Change 90 to your preferred threshold
                stage = 'down'
            elif stage == 'down' and arm_angle > 170:  # Change 170 to your preferred threshold
                stage = 'up'
                counter += 1

        except:
            pass

        # Render detections and display pushup count
        mp.solutions.drawing_utils.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter),
                    (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Display Stage
        cv2.putText(image, 'STAGE', (65, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, stage,
                    (60, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Display the processed image

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


## Code for jumping Jacks

import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Function to check if arms are raised above shoulders
def arms_raised(wrist_y, shoulder_y):
    return wrist_y < shoulder_y

# Function to check if legs are spread wider than hips
def legs_spread(ankle_y, hip_y):
    return ankle_y < hip_y

# Initialize variables for jumping jack counting
counter = 0
stage = 'closed'  # Start with legs and arms closed

# Capture video from camera
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Camera could not be opened.")
else:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Error: Frame not captured.")
            break

        # Flip the frame horizontally and convert to RGB
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Convert back to BGR for OpenCV
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates for wrists, shoulders, ankles, and hips
            left_wrist_y = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
            right_wrist_y = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y
            left_shoulder_y = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            right_shoulder_y = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y
            left_ankle_y = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
            right_ankle_y = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y
            left_hip_y = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
            right_hip_y = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y

            # Determine if jumping jack is in open or closed state
            if arms_raised(left_wrist_y, left_shoulder_y) and arms_raised(right_wrist_y, right_shoulder_y) and legs_spread(left_ankle_y, left_hip_y) and legs_spread(right_ankle_y, right_hip_y):
                if stage == 'closed':
                    stage = 'open'
            else:
                if stage == 'open':
                    stage = 'closed'
                    counter += 1  # Count a jumping jack

        except:
            pass

        # Render pose landmarks and display jumping jack count
        mp.solutions.drawing_utils.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter),
                    (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Display the processed image
        cv2.imshow('Mediapipe Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


# Code for sit ups

import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

# Initialize variables for sit-up counting
counter = 0
stage = 'down'  # Start in the lying down position

# Capture video from camera
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Camera could not be opened.")
else:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Error: Frame not captured.")
            break

        # Flip the frame horizontally and convert to RGB
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Convert back to BGR for OpenCV
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates for shoulders and hips
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            # Calculate shoulder-hip angle
            angle = calculate_angle(left_shoulder, left_hip, right_hip)

            # Sit-up detection logic
            if angle < 45 and stage == 'down':  # Adjust the angle threshold as needed
                stage = 'up'
            elif angle > 70 and stage == 'up':  # Adjust the angle threshold as needed
                stage = 'down'
                counter += 1  # Count a sit-up

        except:
            pass

        # Render pose landmarks and display sit-up count
        mp.solutions.drawing_utils.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter),
                    (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Display the processed image
        cv2.imshow('Mediapipe Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


# For Skipping Rope

import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Function to check if key landmarks are visible
def full_body_visible(landmarks):
    required_landmarks = [mp_pose.PoseLandmark.LEFT_ANKLE, mp_pose.PoseLandmark.RIGHT_ANKLE,
                          mp_pose.PoseLandmark.LEFT_WRIST, mp_pose.PoseLandmark.RIGHT_WRIST]
    return all(landmarks[landmark.value].visibility > 0.5 for landmark in required_landmarks)

# Function to determine if a jump is occurring
def is_jumping(ankles, prev_ankles, wrists, prev_wrists, jump_threshold=0.01, hand_threshold=0.01):
    # Check significant vertical movement of ankles and wrists
    ankles_moved_up = (prev_ankles - ankles) > jump_threshold
    wrists_moved_down = (wrists - prev_wrists) > hand_threshold
    return ankles_moved_up and wrists_moved_down

# Initialize variables for skipping rope counting
counter = 0
prev_ankles = 0
prev_wrists = 0
start_counting = False

# Capture video from camera
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Camera could not be opened.")
else:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Error: Frame not captured.")
            break

        # Flip the frame horizontally and convert to RGB
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Convert back to BGR for OpenCV
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Start counting only when the full body is visible
            if not start_counting and full_body_visible(landmarks):
                start_counting = True

            if start_counting:
                # Get coordinates for ankles and wrists
                left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
                right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y
                left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
                right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y
                ankles = (left_ankle + right_ankle) / 2
                wrists = (left_wrist + right_wrist) / 2

                # Skipping rope detection logic
                if is_jumping(ankles, prev_ankles, wrists, prev_wrists):
                    counter += 1  # Count a jump

                prev_ankles = ankles
                prev_wrists = wrists

        except:
            pass

        # Render pose landmarks and display skipping rope count
        mp.solutions.drawing_utils.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'JUMPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter),
                    (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Display the processed image
        cv2.imshow('Mediapipe Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

#

# Boxing

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Function to calculate the speed of the hand movement
def calculate_speed(current_pos, prev_pos, time_elapsed):
    distance = np.sqrt((current_pos.x - prev_pos.x) ** 2 + (current_pos.y - prev_pos.y) ** 2)
    speed = distance / time_elapsed
    return speed

# Initialize variables
prev_left_wrist = None
prev_right_wrist = None
prev_time = 0
counter = 0
speed_threshold = 0.1  # Adjust this threshold based on testing

# Capture video from camera
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Camera could not be opened.")
else:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Error: Frame not captured.")
            break

        current_time = cv2.getTickCount()
        time_elapsed = (current_time - prev_time) / cv2.getTickFrequency()
        prev_time = current_time

        # Flip the frame horizontally and convert to RGB
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Convert back to BGR for OpenCV
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Get current wrist positions
            current_left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
            current_right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]

            if prev_left_wrist and prev_right_wrist:
                # Calculate speed of each wrist
                left_speed = calculate_speed(current_left_wrist, prev_left_wrist, time_elapsed)
                right_speed = calculate_speed(current_right_wrist, prev_right_wrist, time_elapsed)

                # Check if either hand has punched
                if left_speed > speed_threshold or right_speed > speed_threshold:
                    counter += 1

            prev_left_wrist = current_left_wrist
            prev_right_wrist = current_right_wrist

        except:
            pass

        # Render pose landmarks and display punch count
        mp.solutions.drawing_utils.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.rectangle(image, (0, 0), (300, 100), (245, 117, 16), -1)
        cv2.putText(image, 'PUNCHES', (15, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(image, str(counter),
                    (15, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Display the processed image
        cv2.imshow('Mediapipe Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


# Plank

import cv2
import mediapipe as mp
import numpy as np
import time

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    return angle

# Function to check if the user is in a plank position
def is_in_plank(shoulder, hip, ankle):
    angle = calculate_angle(shoulder, hip, ankle)
    return 160 <= angle <= 200  # Adjust this range as needed

# Initialize variables for plank detection
plank_start_time = 0
plank_duration = 0
in_plank_position = False

# Capture video from camera
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Camera could not be opened.")
else:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Error: Frame not captured.")
            break

        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            # Check plank position
            if is_in_plank(shoulder, hip, ankle):
                if not in_plank_position:
                    in_plank_position = True
                    plank_start_time = time.time()
                plank_duration = time.time() - plank_start_time
            else:
                in_plank_position = False

        except:
            pass

        mp.solutions.drawing_utils.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Display plank duration
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'PLANK TIME', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(int(plank_duration)) + 's',
                    (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()




In [ ]:
# Load the saved model
model = load_model('final_model.h5')